In [4]:
import itertools
import math

def shapley_values(model, instance, features):
    """
    Compute Shapley values for a given model and instance.

    model: callable taking a dict of feature→value, returning a scalar prediction
    instance: dict of actual feature values for the point being explained
    features: list of feature names
    """
    n = len(features)
    phi = {f: 0.0 for f in features}
    fact = math.factorial

    for feat in features:
        others = [f for f in features if f != feat]
        for r in range(len(others) + 1):
            for subset in itertools.combinations(others, r):
                subset = list(subset)
                # value without feat
                base = {f: instance[f] if f in subset else 0 for f in features}
                v_base = model(base)
                # value with feat added
                with_feat = {**base, feat: instance[feat]}
                v_with = model(with_feat)
                weight = fact(len(subset)) * fact(n - len(subset) - 1) / fact(n)
                phi[feat] += weight * (v_with - v_base)
    return phi

# Scenario 1: Simple additive model

Result: Shapley values {f1=3, f2=10}, because each feature's contribution equals its coefficient times its value.

In [12]:
def scenario1_model(data):
    return 3 * data['f1'] + 5 * data['f2']

instance1 = {'f1': 1, 'f2': 2}
features1 = ['f1', 'f2']
sv1 = shapley_values(scenario1_model, instance1, features1)
print("Shapley values:", sv1)

Shapley values: {'f1': 3.0, 'f2': 10.0}


# Scenario 2: One interaction model

Result: Shapley values {f1=1, f2=3, f3=3}; f1 contributes its standalone value 1, while the product term 6 is split evenly between f2 and f3.

In [13]:
def scenario2_model(data):
    return data['f1'] + data['f2'] * data['f3']

instance2 = {'f1': 1, 'f2': 2, 'f3': 3}
features2 = ['f1', 'f2', 'f3']
sv2 = shapley_values(scenario2_model, instance2, features2)
print("Shapley values:", sv2)

Shapley values: {'f1': 1.0, 'f2': 3.0, 'f3': 3.0}


# Scenario 3: Multiple interaction model

Result: Shapley values {f1=1, f2=4, f3=3}; f1 gets half of 2 (→1), f3 half of 6 (→3), and f2 collects the other halves (1+3=4).

In [14]:
def scenario3_model(data):
    return data['f1'] * data['f2'] + data['f2'] * data['f3']

instance3 = {'f1': 1, 'f2': 2, 'f3': 3}
features3 = ['f1', 'f2', 'f3']
sv3 = shapley_values(scenario3_model, instance3, features3)
print("Shapley values:", sv3)

Shapley values: {'f1': 1.0, 'f2': 4.0, 'f3': 3.0}
